In [5]:
import pandas as pd
import re

In [6]:
original_df = pd.read_csv(r'..\results\2-21-25_Scrape.csv')
fice_table = pd.read_csv(r'..\results\fice_table.csv')

In [7]:
# extract and store all extra courses from records that have multiple courses

multi_courses = original_df[original_df['NEUCourse'].str.contains(" and [A-Z]{2,}\s*\d{4}\s-.*", regex=True)]
match_list = []

for i in multi_courses.index:
    row = original_df.loc[i].copy()
    course_names_str = original_df.at[i, 'NEUCourse']
    matches = re.findall(r"and [A-Z]{2,}\s*\d{4}\s-.*?(?=and [A-Z]{2,}\s*\d{4}\s-.*|\Z)", course_names_str)
    matches = [match.strip()[4:] for match in matches]
    for match in matches:
        row["NEUCourse"] = match
        # print(row)
        match_list.append(row)
multi_courses_df = pd.DataFrame(match_list)
# multi_courses_df

In [8]:
# stip away multi courses that were already extracted before

def remove_multi_courses(row):
    row['NEUCourse'] = re.sub(r"and [A-Z]{2,}\s*\d{4}\s-.*\Z", "", row['NEUCourse']).strip()
    return row
multi_courses_removed = original_df.apply(remove_multi_courses, axis=1)
# multi_courses_removed

In [9]:
combined = pd.concat([multi_courses_removed, multi_courses_df])
# combined

In [10]:
valid_courses = combined[combined['NEUCourse'].str.contains(r'[A-Z]+\s*\d{4}', regex=True)]
# valid_courses
# print(valid_courses['NEUCourse'].unique())

In [13]:
# Extract codes using vectorized string operations
codes_out = valid_courses.copy()
codes_out['NEUCode'] = (valid_courses['NEUCourse'].str.extract(r'([A-Z]+\s*\d{4})'))
codes_out['NEUCode'] = codes_out['NEUCode'].str.replace(r'\s+', '', regex=True)
# codes_out
codes_out.to_csv(r'../results/database.csv', index=False)